# 1-Generate Predictions using LangChain

- **Goal:** Prediction Recognition

- **Purpose:** To implement step 1 with sub steps of prediction recognition pipeline. See steps
    1. Generate predictions
        1. Create several prediction prompts templates
        2. Utilize open-source LLMs to generate predictions

- **Misc:**
    - `%store`: Cell magic will store the variable of interest so we can load in another notebook

In [1]:
import os
import sys

import pandas as pd

from langchain_core.prompts import PipelinePromptTemplate, PromptTemplate

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from pipelines import BasePipeline

In [2]:
pd.set_option('max_colwidth', 800)

## LangChain Templates

In [3]:
full_prediction_template = """{prediction_properties}

{prediction_requirements}

{prediction_templates}

{prediction_examples}"""
full_prediction_prompt = PromptTemplate.from_template(full_prediction_template)

In [4]:
prediction_properties_template = """A prediction ($y$) consists of the following eight properties:

    1. $y_p$, {prediction_domain} person that predicted $y$
        - Can be a person (with a name) or a {prediction_domain} person such as a {prediction_domain} reporter, {prediction_domain} analyst, {prediction_domain} expert, {prediction_domain} top executive, {prediction_domain} senior level person, etc).
    2. $y_o$, {prediction_domain} organization 
        - Can only be an organization or entity that is associated with the {prediction_domain} prediction.
    3. $y_t$, current time when $y$ was made
        - Time is the exact moment that can be measured in day, hour, minute, second, etc.
    4. $y_f$, forecast time when $y$ is expected to come to fruition
        - Forecast can be from a second to anytime in the future.
        - Answers the questions: "How far to go out?" or "Where to stop?".
    5. $y_a$, {prediction_domain} prediction attribute
        - Measurable domain-specific attributes such as various quantifiable metrics relevant to the {prediction_domain} domain.
        - Some examples are {prediction_domain_attribute}.
    6. $y_s$, slope that indicates the direction of change in $y_a$
        - Change of directions can be rise/increase/as much as, fall/decrease/as little as, change, stay stable, high/low chance/probability/degree of.
    7. $y_m$, metric outcome
        - How much will the $y_a$ $y_s$?
    8. $y_v$, future verb tense
        - A verb that is associated with the future such as will, would, be going to, should, etc.
    9. $y_l$, location
        - The location is attached to attribute $y_a$ if {prediction_domain} == 'weather'
    """
prediction_properties_prompt = PromptTemplate.from_template(prediction_properties_template)

In [5]:
prediction_requirements_template = """{prediction_domain} requirements to use for each prediction:

    - Should be based on real-world {prediction_domain} data and not hallucinate.
    - Only a simple sentence (prediction) (and NOT compounding using "and" or "or").
    - Should diversify all eight properties of the prediction ($y$).
    - Should use synonyms of predicts such as forecasts, speculates, foresee, envision, etc., and not use any of them more than ten times.
    - The prediction should be unique and not repeated.
    - The forecast time ($y_f$) should always be after current time ($y_t$) of when forecast ($y$) was made.
    - Do not number the predictions.
    - Do not say, "As the {prediction_domain} at organization ($y_o$), I will generate company-based {prediction_domain} predictions using the provided templates." or anything similar.
    - Should have a forecast time ($y_f$) when $y$ is expected to come to fruition between 2025 to 2050.
    - Use the five different templates and examples provided.
    - Change how the current time ($y_t$) and forecast time ($y_f$) are written in the prediction with examples of (1) Wednesday, August 21, 2024; (2) Wed, August 21, 2024; (3) 08/21/2024; (4) 08/21/2024; (5) 21/08/2024; (6) 21 August 2024; (7) 2024/08/21; (8) 2024-08-21; (9) August 21, 2024; (10) Aug 21, 2024; (11) 21 August 2024, (12) 21 Aug 2024, Q3 of 2027, 2029 of Q3, etc (with removing day of week).
    {domain_requirements}
    - Do not say, "Here are 10 unique weather predictions based on the provided templates and examples:" in the prompt."""
prediction_requirements_prompt = PromptTemplate.from_template(prediction_requirements_template)

In [6]:
prediction_templates_template = """Here are some {prediction_domain} templates:

- {prediction_domain} template 1: On [ $y_t$ ], [ $y_p$ ] predicts that the [ $y_a$ ] at [ $y_o$ ] [ $y_v$ ] [ $y_s$ ] by [ $y_m$ ] in [ $y_f$ ].
- {prediction_domain} template 2: In [ $y_t$ ], [ $y_p$ ] from [ $y_o$ ], predicts that the [ $y_a$ ] [ $y_v$ ] [ $y_s$ ] from [ $y_m$ ] in [ $y_f$ ].
- {prediction_domain} template 3: [ $y_p $] predicts on [ $y_t$ ] that the [ $y_a$ ] at [ $y_o$ ] [ $y_v$ ] [ $y_s$ ] under [ $y_m$ ] in [ $y_f$ ].
- {prediction_domain} template 4: According to a [ $y_p$ ] from [ $y_o$ ], on [ $y_t$ ], the [ $y_a$ ] [ $y_v$ ] [ $y_s$ ] beyond [ $y_m$ ] in the timeframe of [ $y_f$ ].
- {prediction_domain} template 5: In [ $y_f$ ], the [ $y_a$ ] at [ $y_o$ ] [ $y_v$ ] a [ $y_m$ ] [ $y_s$ ], as predicted by [ $y_p$ ] on [ $y_t$ ]."""
prediction_templates_prompt = PromptTemplate.from_template(prediction_templates_template)

In [7]:
prediction_examples_template = """Here are some examples of {prediction_domain} predictions:

{domain_examples}


With the above, generate a unique set of {predictions_N} financial predictions. Think from the perspective of an {prediction_domain} anlyst, expert, top executive, or senior level person."""
prediction_examples_prompt = PromptTemplate.from_template(prediction_examples_template)

## Financial

In [8]:
financial_attributes = """stock price, net profit, revenue, operating cash flow, research and development expenses, operating income, gross profit."""
financial_requirements = """- Should be based on real-world financial earnings reports.
    - Suppose the time when $y$ was made is during any earning season.
    - Include stocks from all sectors such as consumer staples, energy, finance, health care, industrials, materials, media, real estate, retail, technology, utilities, defense, etc."""

# For each template, have a rise, fall, or stable example, respectively.
financial_examples = """
- financial examples for template 1:
    1. On [Monday, December 16, 2024], [Detravious, an investor] forecasts that the [revenue] at [Apple] [will likely] [decrease] from [$87B to $50 billion] in [2025 Q1].
    2. On [Tue, November 19, 2024], [Ava Lee] predicts that the [operating cash flow] at [ExxonMobil] [should] [decrease] by [5 percent to $20 billion] in [08/21/2025].
- financial examples for template 2:
    3. In [October 2024], [Julian Hall] from [Yahoo Finance], envisions that the [stock price] [will] [rise] from [$800 to $1,000 per share] in [Quarter 3 of 2028].
    4. In [8/15/2027], [Mrs. Kalia] from [McDonald's], predicts that the [net profit] [will] [fall] under [5% to $5 billion] in [January of 2029].
- financial examples for template 3:
    5. [Dija Gabe, a financial expert] predicts on [23 October 2024] that the [research and development expenses] at [Alphabet] [may] [stay stable] at [$20 million] in [2027 Quarter 4].
    6. [Mr. Mike] predicts in [Q2 2026] that the [operating income] at [Microsoft] [will] [fall] by [407 percent to $50M] on [Monday, Nov 18, 2026].
- financial examples for template 4:
    7. According to a [top executive] from [Chevron], on [08/21/2024], the [net profit] [is expected to] [increase] beyond [$10,000] in the timeframe of [Q3 of 2029].
    8. According to [Brittany] from [Tesla], on [Fri, July 12, 2024], the [gross profit] [may] [increase] as much as [$30,000,000, reflecting a 1209 percent increase] by [21 Aug 2024].
- financial examples for template 5:
    9. In [2025-08-21], the [net profit] at [Amazon] has a [probability] of [11 percent to reach $30k] [decrease], as predicted by [Emily Davis, a financial reporter] on [21 Oct 24].
    10. In [Quarter of 2027], the [revenue] at [Facebook] [is expected] to be [$30 billion, which is a 15%] [rise], as predicted by [a financial analyst] on [Sun, February 20, 2024]."""

## Weather

In [10]:
weather_prediction_templates_template = """Here are some {prediction_domain} templates:

- {prediction_domain} template 1: On [ $y_t$ ], [ $y_p$ ] predicts that the [ $y_a$ ] at [ $y_o$ ] in [ $y_l$ ] [ $y_v$ ] [ $y_s$ ] by [ $y_m$ ] in [ $y_f$ ].
- {prediction_domain} template 2: In [ $y_t$ ], [ $y_p$ ] from [ $y_o$ ] in [ $y_l$ ], predicts that the [ $y_a$ ] [ $y_v$ ] [ $y_s$ ] from [ $y_m$ ] in [ $y_f$ ].
- {prediction_domain} template 3: [ $y_p $] predicts on [ $y_t$ ] that the [ $y_a$ ] at [ $y_o$ ] in [ $y_l$ ] [ $y_v$ ] [ $y_s$ ] under [ $y_m$ ] in [ $y_f$ ].
- {prediction_domain} template 4: According to a [ $y_p$ ] from [ $y_o$ ] in [ $y_l$ ], on [ $y_t$ ], the [ $y_a$ ] [ $y_v$ ] [ $y_s$ ] beyond [ $y_m$ ] in the timeframe of [ $y_f$ ].
- {prediction_domain} template 5: In [ $y_f$ ], the [ $y_a$ ] at [ $y_o$ ] in [ $y_l$ ] [ $y_v$ ] a [ $y_m$ ] [ $y_s$ ], as predicted by [ $y_p$ ] on [ $y_t$ ]."""
weather_prediction_templates_prompt = PromptTemplate.from_template(weather_prediction_templates_template)

In [11]:
weather_attributes = """temperature, precipitation, wind speed, humidity, etc."""
weather_requirements = """- Should be based on real-world weather reports.
    - Suppose the time when $y$ was made is during any season and any location (ie: Florida known for hurricanes, California known for wildfires, etc).
    - Include reports from all meteorologists, weather organizations, or any type of weather entity.."""

# For each template, have a rise, fall, or stable example, respectively.?
weather_examples = """
- weather examples for template 1:
    1. On [Monday, December 16, 2024], [Dr. Melissa Carter] a weather expert at the [National Weather Service], forecasts that the [temperature], in [New York City] [will likely] [decrease] from [5°C to 3°C] on [February 16, 2025 (Fri)].
    2. On [Tue, 19 November 2024], [Ethan James] at the [US Weather Center] predicts that the [precipitation levels], in [San Francisco] [are likely to] [increase] by [20%] in the timeframe of [08/21/2025].
- weather examples for template 2:
    3. In [October 2024], [Samantha Lin] from [NOAA], envisions that the [wind speed] [should] [decrease] by [15 mph] in [Chicago] by [Friday, March 22, 2025].
    4. In [8/15/2027], [Carlos Rivera] from [Weather.com] predicts that the [humidity] [will] [rise] by [30%] in [Miami] in [July of 2025].
- weather examples for template 3:
    5. [Amanda Green], a weather reporter from [Bureau of Meteorology]  predicts on [23 October 2024] that the [temperature] in [Seattle], [will] [fall] by [10°F] in [2025 Quarter 1].
    6. [Mr. Tommy Wu], from [US Weather Center] predicts in [Q2 2026] that [snowfall levels], in [Denver] [will likely] [increase] by [8 inches] in [Monday, Nov 18, 2026].
- weather examples for template 4:
    7. According to a [top executive] from [AccuWeather], on [12/21/2024], the [rainfall] in [Portland] [is expected to] [increase] beyond [10 percent] in the timeframe of [early 2025].
    8. According to [David Harper] from [Weather Underground, on [Fri, August 9, 2024], the [air quality index] in [Los Angeles] [is likely to] [improve] by [20%] in [21 Aug 2024].
- weather examples for template 5:
    9. In [2025-08-21], the [average temperature] in [Houston] has a [probability] of [5 percent to] [decrease], as predicted by [King, a weather reporter] from [Meteorological Department] on [21 Oct 24].
    10. In [Quarter of 2027], wind chill] in [Minneapolis] [is expected] to be [10°F, which is a 15%] [rise], as predicted by [a weather analyst named Ortiz] on [Sun, February 20, 2024]."""

## Health

In [12]:
health_attributes = """obesity rates, prevalence of chronic illnesses, average physical activity levels, nutritional intake, etc."""
health_requirements = """- Should be based on real-world health reports.
    - Suppose the time when $y$ was made is during any season such as flu season, allergy season, pandemic, epidemic, etc.
    - Include reports from all Health organization, researcher, doctor, physical therapist, physician assistant, nurse practictioners, fitness expert, etc."""

# For each template, have a rise, fall, or stable example, respectively.?
health_examples = """
- health examples for template 1:
    1. On [Monday, December 16, 2024], [Detravious, an investor] forecasts that the [obesity rate] at the [United States] [will likely] [decrease] by [5%] in [2025 Q1].
    2. On [Tue, November 19, 2024], [medical professional Sophia Rodriguez] predicts that the [cancer rate] in [Georgia] [should] [decrease] by [4 percent] in [08/21/2025].
- health examples for template 2:
    3. In [October 2024], [Arjun Patel, Ph.D] from [Florida Department of Health] envisions that the [average daily caloric intake] [may] [rise] from [100 to 300] in [Quarter 3 of 2028].
    4. In [8/15/2027], [Dr. Michael Brown] from the [Centers for Disease Control and Prevention], foresee that the [average daily caloric intake] [will] [fall] [8 percent] in [2027]
- Examples for template 3: .
- health examples for template 3:
    5. [A trusted expert] predicts on [23 October 2024] that the [global vaccination rate for measles] in the [US] [should] [stay stable] at [100K people] in [2027 Quarter 4].
    6. [Dr. Sarah Johnson] foresee in [Q2 2026] that the [prevalence of hypertension] in [California] [will] [fall] by [407 percent] by [Monday, Nov 18, 2026].
- health examples for template 4:
    7. According to a [Olivia Martinez] from [Stanford University], on [08/21/2024], the prevalence of [type 2 diabetes in adults] [is expected to] [increase] beyond [8.5 percent] in the timeframe of [Q3 of 2029].
    8. According to [Rachel Kim, MD] from the [University of California], on [Fri, July 12, 2024], the prevalence of [type 2 diabetes in adults] [may] [increase] as much as [30,000,000, reflecting a 1209 percent increase] by [21 Aug 2024].
- health examples for template 5:
    9. In [2025-08-21], the [average weekly exercise hours] in [United States] has a [probability] of [20 percent to reach 30k], as predicted by [Emily Davis, Harvard School of Public Health] on [21 Oct 24].
    10. In [Quarter of 2027], the [average weekly walking hours] in [Atlanta] is [expected to rise] by [15%], as predicted by the [Monique, National Institutes of Health] on [Sun, February 20, 2024]."""

## Policy

In [13]:
policy_attributes = """election outcomes, economic reforms, legislative impacts."""
policy_requirements = """- Should be based on real-world policy reports.
    - Suppose the time when $y$ was made is during an election cycle or non-election cycles.
    - Include policies & laws, from all sectors such as consumer staples, energy, finance, health care, industrials, materials, media, real estate, retail, technology, utilities, defense, etc."""

policy_examples = """
- policy examples for template 1:
    1. On [Monday, December 16, 2024], [President John Doe] forecasts that the [unemployment rate] at [the United States] [will likely] [decrease] by [2%] in [2025 Q1].
    2. On [Tue, November 19, 2024], [Dr. Jane Smith] foresee that the [population growth rate] in [California] [is likely to] [decrease] by [5 percent to 20 billion] in [08/21/2025].
- policy examples for template 2:
    3. In [October 2024], [Senator Emily Johnson] from [the Senate Committee on Finance], envisions that the [inflation rate] [should] [rise] from [1.3 percent to 89 percen] in [Quarter 3 of 2028].
    4. In [8/15/2027], [Governor Michael Brown] from [the State of Texas], predicts that the [number of registered voters] [will] [fall] under [5B] in [Dec of 2029].
- policy examples for template 3:
    5. [Dija Gabe in the Congressional Budget Office] predicts on [23 October 2024] that [national debt] in [USA] [may] [stay stable] at [20 million] in [2027 Quarter 4].
    6. [Dr. Sarah Lee] foresee in [Q2 2026] that the [median household income] in [NY] [should] [fall] by [629 percent to $15,000] on [Monday, Nov 18, 2026].
- policy examples for template 4:
    7. According to a [General Robert Williams] from [the Department of Defense], on [08/21/2024], the [number of active-duty soldiers] [is expected to] [increase] beyond [$10,000] in the timeframe of [Q3 of 2029].
    8. According to [Dr. Olivia Martinez] from [the Census Bureau], on [Fri, July 12, 2024], the [population density] in [urban areas] [is likely to] [increase] as much as [100,000,000, reflecting a 1209 percent increase] by [21 Aug 2024].
- policy examples for template 5:
    9. In [2025-08-21], the [number of citizens] in [Thomson, GA, 30824] has a [probability] of [92 percent to reach 30k] [decrease], as predicted by [Shirly Tisdale, a policy reporter] on [21 Oct 24].
    10. In [Quarter of 2027], the  [number of Navy members] in [the United States] [is expected] to be [300K, which is a 15%] [rise], as predicted by [a policy analyst] on [Sun, February 20, 2024]."""

## Generate Predictions

In [14]:
input_prompts = [
    ("prediction_properties", prediction_properties_prompt),
    ("prediction_requirements", prediction_requirements_prompt),
    ("prediction_templates", prediction_templates_prompt),
    ("prediction_examples", prediction_examples_prompt),
]

pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prediction_prompt, pipeline_prompts=input_prompts
)

financial_input_dict = {
    "prediction_domain": "financial",
    "prediction_domain_attribute": financial_attributes,
    "domain_requirements": financial_requirements,
    "domain_examples": financial_examples,
    "predictions_N": 10
}
financial_prompt_output = pipeline_prompt.format(**financial_input_dict)
print(financial_prompt_output)


weather_input_prompts = [
    ("prediction_properties", prediction_properties_prompt),
    ("prediction_requirements", prediction_requirements_prompt),
    ("prediction_templates", weather_prediction_templates_prompt),
    ("prediction_examples", prediction_examples_prompt),
]

weather_pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prediction_prompt, pipeline_prompts=weather_input_prompts
)

weather_input_dict = {
    "prediction_domain": "weather",
    "prediction_domain_attribute": weather_attributes,
    "domain_requirements": weather_requirements,
    "domain_examples": weather_examples,
    "predictions_N": 10
}
weather_prompt_output = weather_pipeline_prompt.format(**weather_input_dict)
print(weather_prompt_output)

health_input_dict = {
    "prediction_domain": "health",
    "prediction_domain_attribute": health_attributes,
    "domain_requirements": health_requirements,
    "domain_examples": health_examples,
    "predictions_N": 10
}

health_prompt_output = pipeline_prompt.format(**health_input_dict)
print(health_prompt_output)

policy_input_dict = {
    "prediction_domain": "policy",
    "prediction_domain_attribute": policy_attributes,
    "domain_requirements": policy_requirements,
    "domain_examples": policy_examples,
    "predictions_N": 10
}

policy_prompt_output = pipeline_prompt.format(**policy_input_dict)
print(policy_prompt_output)

base_pipeline = BasePipeline()
financial_df = base_pipeline.generate_predictions(financial_prompt_output, 1, "financial")
weather_df = base_pipeline.generate_predictions(weather_prompt_output, 1, "weather")
health_df = base_pipeline.generate_predictions(health_prompt_output, 1, "health")
policy_df = base_pipeline.generate_predictions(policy_prompt_output, 1, "policy")

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_39940/714355400.py:8: LangChainDeprecationWarning: This class is deprecated. Please see the docstring below or at the link for a replacement option: https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html
  pipeline_prompt = PipelinePromptTemplate(


A prediction ($y$) consists of the following eight properties:

    1. $y_p$, financial person that predicted $y$
        - Can be a person (with a name) or a financial person such as a financial reporter, financial analyst, financial expert, financial top executive, financial senior level person, etc).
    2. $y_o$, financial organization 
        - Can only be an organization or entity that is associated with the financial prediction.
    3. $y_t$, current time when $y$ was made
        - Time is the exact moment that can be measured in day, hour, minute, second, etc.
    4. $y_f$, forecast time when $y$ is expected to come to fruition
        - Forecast can be from a second to anytime in the future.
        - Answers the questions: "How far to go out?" or "Where to stop?".
    5. $y_a$, financial prediction attribute
        - Measurable domain-specific attributes such as various quantifiable metrics relevant to the financial domain.
        - Some examples are stock price, net prof

In [15]:
financial_df

,Base Sentences,Prediction Label,Model Name,Domain
0,"On 2024-10-15, Olivia Brown, a financial analyst, predicts that the operating income at General Motors will likely rise by 10 percent to $5 billion in Q2 of 2026.",1,llama-3.3-70b-versatile,financial
1,"In 2024/08/20, Ethan Kim from Goldman Sachs, forecasts that the stock price will increase from $500 to $700 per share in 2028 Q4.",1,llama-3.3-70b-versatile,financial
2,"According to a senior executive from Boeing, on August 22, 2024, the revenue is expected to increase beyond $100 billion in the timeframe of 2029 Q3.",1,llama-3.3-70b-versatile,financial
3,"On 21 Aug 2024, Ava Morales, a financial expert, predicts that the research and development expenses at Pfizer will stay stable at $15 million in 2027 Q1.",1,llama-3.3-70b-versatile,financial
4,"In Q3 of 2025, the net profit at Visa will fall by 5 percent to $10 billion, as predicted by David Lee, a financial reporter, on 2024-10-10.",1,llama-3.3-70b-versatile,financial
5,"On 2024/11/12, Emily Patel, a financial senior level person, predicts that the gross profit at Mastercard will rise by 20 percent to $8 billion in 2026 Q2.",1,llama-3.3-70b-versatile,financial
6,"In 2025-11-15, the operating cash flow at Intel will have a high chance of reaching $25 billion, which is a 10 percent increase, as predicted by Jackson Hall, a financial analyst, on 2024/08/25.",1,llama-3.3-70b-versatile,financial
7,"On 2024/10/18, Michael Davis, a top executive from Cisco, predicts that the revenue will decrease by 8 percent to $40 billion in Q4 of 2027.",1,llama-3.3-70b-versatile,financial
8,"In 2027 Q1, the stock price at Netflix will be $900 per share, which is a 15 percent rise, as predicted by Sophia Rodriguez, a financial expert, on 2024/09/20.",1,llama-3.3-70b-versatile,financial
9,"According to a financial analyst from IBM, on 2024-08-28, the operating income is expected to increase beyond $6 billion in the timeframe of 2030 Q2.",1,llama-3.3-70b-versatile,financial


In [16]:
weather_df

,Base Sentences,Prediction Label,Model Name,Domain
0,"On 2024-10-15, Dr. Rachel Lee, a senior meteorologist at the National Weather Service, forecasts that the precipitation levels in Los Angeles will likely increase by 15% in Q2 of 2026.",1,llama-3.3-70b-versatile,weather
1,"In 2024/08/20, Ethan Hall, a weather expert from the European Centre for Medium-Range Weather Forecasts, predicts that the wind speed in London will decrease by 10 mph in 2027-03-15.",1,llama-3.3-70b-versatile,weather
2,"According to a weather analyst from the Australian Bureau of Meteorology, on 2024-09-10, the temperature in Sydney will rise by 5°C in the timeframe of 2025-01-01.",1,llama-3.3-70b-versatile,weather
3,"On 08/22/2024, Michael Patel, a weather reporter from the India Meteorological Department, predicts that the humidity in Mumbai will fall by 20% in 2026-06-01.",1,llama-3.3-70b-versatile,weather
4,"In 2025-10-12, the average temperature in Beijing has a probability of 10% to decrease, as predicted by Dr. Liu, a weather expert from the China Meteorological Administration, on 2024-11-05.",1,llama-3.3-70b-versatile,weather
5,"On 2024-07-25, Dr. Maria Rodriguez, a top executive from the Spanish Meteorological Agency, forecasts that the snowfall levels in Madrid will increase by 12 inches in Q1 of 2028.",1,llama-3.3-70b-versatile,weather
6,"In 2024/12/18, David Kim, a weather analyst from the Korean Meteorological Administration, predicts that the air quality index in Seoul will improve by 15% in 2026-09-20.",1,llama-3.3-70b-versatile,weather
7,"According to a senior level meteorologist from the Canadian Meteorological Centre, on 2024-06-15, the rainfall in Vancouver will increase beyond 15% in the timeframe of 2027-04-01.",1,llama-3.3-70b-versatile,weather
8,"On 2024-05-20, Dr. John Taylor, a weather expert from the UK Met Office, predicts that the wind chill in Edinburgh will rise by 8°F in 2026-02-22.",1,llama-3.3-70b-versatile,weather
9,"In Q4 of 2029, the precipitation levels in Tokyo will likely decrease by 10%, as predicted by Yui Nakamura, a weather reporter from the Japan Meteorological Agency, on 2024-03-10.",1,llama-3.3-70b-versatile,weather


In [17]:
health_df

,Base Sentences,Prediction Label,Model Name,Domain
0,"On 2024-10-15, Dr. Thompson, a health expert from the World Health Organization, predicts that the obesity rate at the United States will likely decrease by 3% in Q2 of 2026.",1,llama-3.3-70b-versatile,health
1,"In 2024, Dr. Lee, a researcher from the Centers for Disease Control and Prevention, envisions that the average daily caloric intake may rise from 2000 to 2500 in Quarter 4 of 2028.",1,llama-3.3-70b-versatile,health
2,"Dr. Patel, a trusted health analyst, predicts on 20 August 2024 that the global vaccination rate for influenza in the US should stay stable at 90% in 2027 Quarter 3.",1,llama-3.3-70b-versatile,health
3,"According to a report by Dr. Kim, a health expert from the University of California, on 12 July 2024, the prevalence of type 2 diabetes in adults is expected to increase beyond 9% in the timeframe of Q3 of 2030.",1,llama-3.3-70b-versatile,health
4,"In 2025-08-20, the average weekly exercise hours in Australia has a probability of 25% to reach 25k, as predicted by Dr. Davis, a health expert from the National Institutes of Health, on 15 October 2024.",1,llama-3.3-70b-versatile,health
5,"On Wednesday, 21 August 2024, Dr. Rodriguez, a medical professional, forecasts that the cancer rate in Canada will likely decrease by 6% in 2026 Q1.",1,llama-3.3-70b-versatile,health
6,"In Q2 of 2027, Dr. Brown, a health expert from the Centers for Disease Control and Prevention, predicts that the average daily physical activity levels will fall by 10% in 2027, as part of a larger trend.",1,llama-3.3-70b-versatile,health
7,"Dr. Johnson, a health expert, predicts on 18 November 2024 that the prevalence of hypertension in the UK will fall by 500 cases in 2026 Quarter 2.",1,llama-3.3-70b-versatile,health
8,"According to a report by Dr. Martin, a health expert from the World Health Organization, on 2024-08-21, the average life expectancy is expected to increase beyond 80 years in the timeframe of Q4 of 2035.",1,llama-3.3-70b-versatile,health
9,"In 2029-08-21, the average weekly walking hours in the United States is expected to rise by 20%, as predicted by Dr. Smith, a health expert from the National Institutes of Health, on 10 February 2024.",1,llama-3.3-70b-versatile,health


In [18]:
policy_df

,Base Sentences,Prediction Label,Model Name,Domain
0,"On 2024-10-15, policy expert, Rachel Kim, forecasts that the stock price at Goldman Sachs will likely rise by 15% in 2026 Q2.",1,llama-3.3-70b-versatile,policy
1,"In 2025, policy analyst, David Lee, from the Federal Reserve, predicts that the interest rate will increase from 2% to 5% in 2028 Q4.",1,llama-3.3-70b-versatile,policy
2,"According to a policy senior level person, James Davis, from JPMorgan Chase, on 08/20/2024, the number of mortgage applications is expected to decrease beyond 10,000 in the timeframe of 2029 Q1.",1,llama-3.3-70b-versatile,policy
3,"In 2027-08-22, the revenue at Morgan Stanley will have a high chance of reaching $10 billion, which is a 20% increase, as predicted by policy top executive, Emily Chen, on 2024-08-19.",1,llama-3.3-70b-versatile,policy
4,"On 21 August 2024, policy expert, Michael Brown, predicts that the unemployment rate in the financial sector at Bank of America will likely decrease by 1% in 2025 Q3.",1,llama-3.3-70b-versatile,policy
5,"In Q2 of 2026, policy analyst, Sophia Patel, from the Securities and Exchange Commission, envisions that the number of initial public offerings will rise from 100 to 500 in 2027 Q2.",1,llama-3.3-70b-versatile,policy
6,"On 2024/10/12, policy senior level person, Kevin White, predicts that the GDP growth rate at Wells Fargo will stay stable at 2% in 2028 Q1.",1,llama-3.3-70b-versatile,policy
7,"According to a policy expert, Olivia Taylor, from the Financial Industry Regulatory Authority, on 2024-10-10, the number of financial transactions is expected to increase as much as 50% in the timeframe of 2030 Q2.",1,llama-3.3-70b-versatile,policy
8,"In 2025 Q1, the profit margin at Citigroup will have a probability of 80% to reach 15%, which is a 5% increase, as predicted by policy top executive, Daniel Hall, on 2024-09-15.",1,llama-3.3-70b-versatile,policy
9,"On 2024-08-25, policy analyst, Jessica Martin, predicts that the inflation rate in the financial sector at UBS will likely rise by 2% in 2026 Q4.",1,llama-3.3-70b-versatile,policy


## Misc

- Need to optimize code
- Include multiple models next

In [ ]:
# domains = ["financial", "weather", "health care"]
# llama_model = LlamaTextGenerationModel()
# model_2 = SomeModel()

# # Should all model dfs be a list of dataframes? Then we can concatenate them all together with pd.concat()
# for domain in domains:
#     llama_df = llama_model.generate_predictions(financial_prompt_output, 1, domain)
#     model_2_df = model_2.generate_predictions(financial_prompt_output, 1, domain)


# # or 

# llama_df = llama_model.generate_predictions(financial_prompt_output, 1, "financial")
# model_2_df = model_2.generate_predictions(financial_prompt_output, 1, "financial")

# llama_df = llama_model.generate_predictions(financial_prompt_output, 1, "weather")
# model_2_df = model_2.generate_predictions(financial_prompt_output, 1, "weather")

# llama_df = llama_model.generate_predictions(financial_prompt_output, 1, "health care")
# model_2_df = model_2.generate_predictions(financial_prompt_output, 1, "health care")